# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
# GLOBAL VARIABLE
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
    
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
populationMap = (cities[['Metropolitan area', 'Population (2016 est.)[8]']]
 .set_index('Metropolitan area')
 .to_dict()['Population (2016 est.)[8]'])

In [2]:
# Check whether any teamToCityMap is incomplete
# for cmp in compe_list:
#     if cmp not in set(teamToCityMap.keys()):
#         print(cmp)

In [3]:
def nhl():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    df = nhl_df.copy()
    comp = 'NHL'
    df = df[df['year'] == 2018]
    df = df[df['year'] == 2018]
    cities[comp] = cities[comp].apply(lambda x: re.sub(r'\d+|\*|\[[^)]*\]|—', '', x))
    cities[comp] = cities[comp].apply(lambda x: x.strip())
    team_list = [team for team in cities[comp].unique() if team != '']

    df['team'] = df['team'].apply(lambda x: re.sub(r'\d+|\*|\[[^)]*\]|—', '', x))
    df['team'] = df['team'].apply(lambda x: x.strip())
    compe_list = [team for team in df['team'].unique() if team != '']

    teamMap = {}
    for team in team_list:
        for cmp in compe_list:
            if team in cmp:
                teamMap.update({cmp: team})

    cityMap = {}
    for i in range(cities.shape[0]):
        if (cities[comp].loc[i] != ''):
            cityMap.update({cities[comp].loc[i]: cities['Metropolitan area'].loc[i]})

    indexes = []
    for i in range(df.shape[0]):
        if not('Division' in df['team'].loc[i]):
            indexes.append(i)
    df = df.loc[indexes]

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])

    teamToCityMap = {}
    for key, values in teamMap.items():
        teamToCityMap.update({key: cityMap[values]})

    teamToCityMap.update({'New York Islanders': 'New York City'})
    teamToCityMap.update({'New York Rangers': 'New York City'})
    teamToCityMap.update({'New Jersey Devils': 'New York City'})
    teamToCityMap.update({'Los Angeles Kings': 'Los Angeles'})
    teamToCityMap.update({'Anaheim Ducks':'Los Angeles'})

    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    populationMap = (cities[['Metropolitan area', 'Population (2016 est.)[8]']]
     .set_index('Metropolitan area')
     .to_dict()['Population (2016 est.)[8]'])

    proc_df = df.copy()
    proc_df['city'] = proc_df['team'].map(teamToCityMap)
    proc_df['pops'] = proc_df['city'].map(populationMap)
    proc_df = proc_df.groupby('city').sum()
    proc_df['W/L'] = proc_df['W'] / (proc_df['W']+proc_df['L'])
    proc_df = proc_df.reset_index()[['city', 'pops', 'W/L']]

    return proc_df

def nhl_correlation(): 
    proc_df = nhl()
    WLMap = proc_df[['city', 'W/L']].set_index('city').to_dict()['W/L']
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    for key in populationMap.keys():
        if key in proc_df['city'].unique():
            population_by_region.append(populationMap[key])
            win_loss_by_region.append(WLMap[key])
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [4]:
nhl_correlation()

0.012308996455744285

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
def nba():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    df = nba_df.copy()
    comp = 'NBA'
    df = df[df['year'] == 2018]
    cities[comp] = cities[comp].apply(lambda x: re.sub(r'\d+|\*|\[[^)]*\]|—', '', x))
    cities[comp] = cities[comp].apply(lambda x: x.strip())
    team_list = [team for team in cities[comp].unique() if team != '']

    df['team'] = df['team'].apply(lambda x: re.sub(r'\d+|\*|\([^)]*\)|—', '', x))
    df['team'] = df['team'].apply(lambda x: x.strip())
    comp_list = [team for team in df['team'].unique() if team != '']

    teamMap = {}
    for team in team_list:
        for cmp in comp_list:
            if team in cmp:
                teamMap.update({cmp: team})

    cityMap = {}
    for i in range(cities.shape[0]):
        if (cities[comp].loc[i] != ''):
            cityMap.update({cities[comp].loc[i]: cities['Metropolitan area'].loc[i]})

    indexes = []
    for i in range(df.shape[0]):
        if not('Division' in df['team'].loc[i]):
            indexes.append(i)
    df = df.loc[indexes]

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])

    teamToCityMap = {}
    for key, values in teamMap.items():
        teamToCityMap.update({key: cityMap[values]})

    teamToCityMap.update({"New York Knicks": "New York City"})
    teamToCityMap.update({"Brooklyn Nets": "New York City"})
    teamToCityMap['Los Angeles Clippers'] = 'Los Angeles'
    teamToCityMap['Los Angeles Lakers'] = 'Los Angeles'

    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    populationMap = (cities[['Metropolitan area', 'Population (2016 est.)[8]']]
     .set_index('Metropolitan area')
     .to_dict()['Population (2016 est.)[8]'])

    proc_df = df.copy()
    proc_df['city'] = proc_df['team'].map(teamToCityMap)
    proc_df['pops'] = proc_df['city'].map(populationMap)
    proc_df = proc_df.groupby('city').sum()
    proc_df['W/L'] = proc_df['W'] / (proc_df['W']+proc_df['L'])
    proc_df = proc_df.reset_index()[['city', 'pops', 'W/L']]
    
    return proc_df

def nba_correlation():
    proc_df = nba()
    WLMap = proc_df[['city', 'W/L']].set_index('city').to_dict()['W/L']

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    for key in populationMap.keys():
        if key in proc_df['city'].unique():
            population_by_region.append(populationMap[key])
            win_loss_by_region.append(WLMap[key])

    stats.pearsonr(population_by_region, win_loss_by_region)[0]
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [6]:
nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [7]:
def mlb():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    df = mlb_df.copy()
    comp = 'MLB'
    df = df[df['year'] == 2018]
    cities[comp] = cities[comp].apply(lambda x: re.sub(r'\d+|\*|\[[^)]*\]|—', '', x))
    cities[comp] = cities[comp].apply(lambda x: x.strip())
    team_list = [team for team in cities[comp].unique() if team != '']

    df['team'] = df['team'].apply(lambda x: re.sub(r'\d+|\*|\([^)]*\)|—', '', x))
    df['team'] = df['team'].apply(lambda x: x.strip())
    compe_list = [team for team in df['team'].unique() if team != '']

    teamMap = {}
    for team in team_list:
        for cmp in compe_list:
            if team in cmp:
                teamMap.update({cmp: team})

    cityMap = {}
    for i in range(cities.shape[0]):
        if (cities[comp].loc[i] != ''):
            cityMap.update({cities[comp].loc[i]: cities['Metropolitan area'].loc[i]})

    indexes = []
    for i in range(df.shape[0]):
        if not('Division' in df['team'].loc[i]):
            indexes.append(i)
    df = df.loc[indexes]

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])

    teamToCityMap = {}
    for key, values in teamMap.items():
        teamToCityMap.update({key: cityMap[values]})

    teamToCityMap.update({'New York Yankees': 'New York City'})
    teamToCityMap.update({'Chicago White Sox': 'Chicago'})
    teamToCityMap.update({'Oakland Athletics': 'San Francisco Bay Area'})
    teamToCityMap.update({'Los Angeles Angels': 'Los Angeles'})
    teamToCityMap.update({'New York Mets': 'New York City'})
    teamToCityMap.update({'Chicago Cubs': 'Chicago'})
    teamToCityMap.update({'Los Angeles Dodgers': 'Los Angeles'})
    teamToCityMap.update({'San Francisco Giants': 'San Francisco Bay Area'})

    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    populationMap = (cities[['Metropolitan area', 'Population (2016 est.)[8]']]
     .set_index('Metropolitan area')
     .to_dict()['Population (2016 est.)[8]'])

    proc_df = df.copy()
    proc_df['city'] = proc_df['team'].map(teamToCityMap)
    proc_df['pops'] = proc_df['city'].map(populationMap)
    proc_df = proc_df.groupby('city').sum()
    proc_df['W/L'] = proc_df['W'] / (proc_df['W']+proc_df['L'])
    proc_df = proc_df.reset_index()[['city', 'pops', 'W/L']]
    
    return proc_df

def mlb_correlation(): 
    proc_df = mlb()
    WLMap = proc_df[['city', 'W/L']].set_index('city').to_dict()['W/L']

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    for key in populationMap.keys():
        if key in proc_df['city'].unique():
            population_by_region.append(populationMap[key])
            win_loss_by_region.append(WLMap[key])
        
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [8]:
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [9]:
def nfl():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    df = nfl_df.copy()
    comp = 'NFL'
    df = df[df['year'] == 2018]
    cities[comp] = cities[comp].apply(lambda x: re.sub(r'\d+|\*|\[[^)]*\]|—', '', x))
    cities[comp] = cities[comp].apply(lambda x: x.strip())
    team_list = [team for team in cities[comp].unique() if team != '']

    df['team'] = df['team'].apply(lambda x: re.sub(r'\d+|[\+\*]|\([^)]*\)', '', x))
    df['team'] = df['team'].apply(lambda x: x.strip())
    compe_list = [team for team in df['team'].unique() if team != '']

    teamMap = {}
    for team in team_list:
        for cmp in compe_list:
            if team in cmp:
                teamMap.update({cmp: team})

    cityMap = {}
    for i in range(cities.shape[0]):
        if (cities[comp].loc[i] != ''):
            cityMap.update({cities[comp].loc[i]: cities['Metropolitan area'].loc[i]})

    indexes = []
    for i in range(df.shape[0]):
        if not('AFC' in df['team'].loc[i]) and not('NFC' in df['team'].loc[i]):
            indexes.append(i)
    df = df.loc[indexes]

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])

    teamToCityMap = {}
    for key, values in teamMap.items():
        teamToCityMap.update({key: cityMap[values]})

    teamToCityMap.update({'New York Jets': 'New York City'})
    teamToCityMap.update({'Los Angeles Chargers': 'Los Angeles'})
    teamToCityMap.update({'Oakland Raiders': 'San Francisco Bay Area'})
    teamToCityMap.update({'New York Giants': 'New York City'})
    teamToCityMap.update({'Los Angeles Rams': 'Los Angeles'})
    teamToCityMap.update({'San Francisco ers': 'San Francisco Bay Area'})

    proc_df = df.copy()
    proc_df['city'] = proc_df['team'].map(teamToCityMap)
    proc_df['pops'] = proc_df['city'].map(populationMap)
    proc_df = proc_df.groupby('city').sum()
    proc_df['W/L'] = proc_df['W'] / (proc_df['W']+proc_df['L'])
    proc_df = proc_df.reset_index()[['city', 'pops', 'W/L']]
    
    return proc_df

def nfl_correlation(): 
    proc_df = nfl()
    WLMap = proc_df[['city', 'W/L']].set_index('city').to_dict()['W/L']

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    for key in populationMap.keys():
        if key in proc_df['city'].unique():
            population_by_region.append(populationMap[key])
            win_loss_by_region.append(WLMap[key])

    stats.pearsonr(population_by_region, win_loss_by_region)[0]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [10]:
nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    nhl_proc = nhl()
    nba_proc = nba()
    nfl_proc = nfl()
    mlb_proc = mlb()

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    def pval(df1, df2, col1, col2, pval_df):
        dfmerge = pd.merge(df1, df2, on='city')
        pval_df.loc[col1, col2] = stats.ttest_rel(dfmerge['W/L_x'], dfmerge['W/L_y'])[1]
        pval_df.loc[col2, col1] = pval_df.loc[col1, col2]

    pval(nba_proc, nhl_proc, 'NBA', 'NHL', p_values)
    pval(nba_proc, nfl_proc, 'NBA', 'NFL', p_values)
    pval(nba_proc, mlb_proc, 'NBA', 'MLB', p_values)
    pval(nhl_proc, nfl_proc, 'NHL', 'NFL', p_values)
    pval(nhl_proc, mlb_proc, 'NHL', 'MLB', p_values)
    pval(nfl_proc, mlb_proc, 'NFL', 'MLB', p_values)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values